In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import random
%matplotlib inline

### UN Data

In [2]:
status_features = ['destination', 'Origin','applied', \
                   'accepted', 'Rejected', 'decisions', 'Year']
number_features = ['destination', 'Origin', 'Year', 'Month', 'Value']
unique_features = ['destination', 'Origin', 'applied', 'accepted', 'Rejected', 'decisions',\
                  'Year', 'Month', 'Value']
responses = ['Value']

numeric_features = ['applied', 'accepted', 'Rejected', 'decisions', 'Year', 'Value',\
                   'deaths']

added_features = ['last_month', 'two_months_ago', 'next_month', 'two_months_later']

merged_features= ['destination', 'Origin', 'applied', 'accepted', 'Rejected', 'decisions',\
                  'Year', 'Month', 'Value', 'deaths']

origins = ["DR_Congo", "Afghanistan", "Syria", "Myanmar", "Sudan"]
destinations = ["USA", "UK", "France", "Canada", "Italy", "Germany"]

years = [2000 + i for i in range(18)]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"]

pairs = []
for origin in origins:
    for destination in destinations:
        pairs.append((origin, destination))

all_features = merged_features + added_features

file1 = './data/asylum_seekers_status_yearly.csv'
file2 = './data/asylum_seekers_monthly.csv'
status = pd.read_csv(file1, skipinitialspace=True)
number = pd.read_csv(file2, skipinitialspace=True)

In [3]:
number = number.rename(index=str, columns = {'Country / territory of asylum/residence':'destination'})
status = status.rename(index=str, columns = {'Country / territory of asylum/residence':'destination',\
                                            'statistics.filter.decisions_recognized':'accepted',\
                                            'Applied during year':'applied', 'Total decisions':'decisions'})

status = status[status_features]

status.ix[status.destination == 'United States of America', ['destination']] = 'USA'
status.ix[status.destination == 'United Kingdom', ['destination']] = 'UK'
number.ix[number.destination == 'USA (INS/DHS)', ['destination']] = 'USA'
number.ix[number.destination == 'USA (EOIR)', ['destination']] = 'USA'
number.ix[number.destination == 'United Kingdom of Great Britain and Northern Ireland', ['destination']] = 'UK'
number.ix[number.Origin.str.contains('Syria'), ['Origin']] = 'Syria'
number.ix[number.Origin.str.contains('Dem. Rep. of the Congo'), ['Origin']] = 'DR_Congo'
status.ix[status.Origin.str.contains('Syria'), ['Origin']] = 'Syria'
status.ix[status.Origin.str.contains('Dem. Rep. of the Congo'), ['Origin']] = 'DR_Congo'

/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipyk

In [4]:
for feature in numeric_features:
    if feature in status_features:
        try:
            status.ix[status[feature] == '*'] = 0
        except:
            trash = None
        status[feature] = pd.to_numeric(status[feature])
    if feature in number_features:
        try:
            number.ix[number[feature] == '*'] = 0
        except:
            trash = None
        number[feature] = pd.to_numeric(number[feature])

/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/Users/jasonterry/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


In [5]:
status = status.fillna(0)
status = status[status_features]

In [6]:
for pair in pairs:
    for year in years:
        if year not in number.query('Origin == @pair[0] and destination == @pair[1]').Year.unique():
            for month in months:
                rand = random.random()
                number = number.append({'Year':year, 'Value':rand*1e-16, 'destination':pair[1], 'Month':month, \
                                       'Origin': pair[0]}, ignore_index=True)
        if year not in status.query('Origin == @pair[0] and destination == @pair[1]').Year.unique():
            rand = random.random()
            status = status.append({'Year':year, 'Month':month, 'destination': pair[1], 'Origin':pair[0],\
                                   'applied': 1e-16, 'decisions': rand*1e-16, 'accepted': 1e-16, 'Rejected':1e-16},\
                                  ignore_index=True)
number = number[number_features]
status = status[status_features]

In [7]:
status = status.groupby(['Year', 'Origin', 'destination'])\
                .agg({'applied':np.sum, 'accepted':np.sum, 'Rejected':np.sum, 'decisions':np.sum})\
                .sort_values(by='Year')

In [8]:
number = number.sort_values(by='Year')

In [9]:
merged = pd.merge(number, status, left_on=['destination', 'Origin', 'Year'], \
                  right_on=['destination', 'Origin', 'Year'], how='inner')

In [10]:
final = merged[unique_features]

### Battle Deaths

In [11]:
file_name = './data/battle_deaths.csv'
b_deaths = pd.read_csv(file_name, skipinitialspace=True)

In [12]:
bd_features = ['year', 'deaths', 'battle_location']
b_deaths = b_deaths[bd_features]
b_deaths = b_deaths[b_deaths['year']>1999]

In [13]:
b_deaths.ix[b_deaths.battle_location.str.contains('Syria'), ['battle_location']] = 'Syria'
b_deaths.ix[b_deaths.battle_location.str.contains('Afghanistan'), ['battle_location']] = 'Afghanistan'
b_deaths.ix[b_deaths.battle_location.str.contains('Sudan'), ['battle_location']] = 'Sudan'
b_deaths.ix[b_deaths.battle_location.str.contains('Congo'), ['battle_location']] = 'DR_Congo'
b_deaths.ix[b_deaths.battle_location.str.contains('Zaire'), ['battle_location']] = 'DR_Congo'
b_deaths.ix[b_deaths.battle_location.str.contains('Myanmar'), ['battle_location']] = 'Myanmar'
b_deaths.ix[b_deaths.battle_location.str.contains('Burma'), ['battle_location']] = 'Myanmar';

/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from th

In [14]:
b_deaths = b_deaths.query("battle_location == \'DR_Congo\' \
                            or battle_location == \'Sudan\'\
                            or battle_location == \'Myanmar\'\
                            or battle_location == \'Syria\'\
                            or battle_location == \'Afghanistan\'")

In [15]:
for origin in origins:
    for a_year in years:
        if a_year not in b_deaths.query('battle_location == @origin').year.unique():
            rand = random.random()
            b_deaths = b_deaths.append({'year':a_year, 'deaths':rand*1e-16, 'battle_location':origin}, \
                                       ignore_index=True)
b_deaths = b_deaths[["year", "deaths", "battle_location"]]

In [16]:
b_deaths = b_deaths.groupby(['battle_location', 'year'])\
        .agg({'deaths':np.sum})\
        .sort_values(by='year')

### Merge and group

In [17]:
all_merged = pd.merge(final, b_deaths, left_on=['Origin', 'Year'], \
                  right_on=['battle_location', 'year'])
all_merged = all_merged[merged_features]
all_merged.deaths = pd.to_numeric(all_merged.deaths)
all_merged.to_csv('./data/all_merged.csv')

In [18]:
groups = all_merged.groupby(['Origin', 'destination'])

In [19]:
group_dict = {}
for (name, df) in groups:
    for feature in numeric_features:
        if type(list(df[feature])[0]) is str:
            df.ix[df[feature] == '*', [feature]] = 0
    df = df.fillna(method='pad')
    for feature in numeric_features:
        df[feature] = pd.to_numeric(df[feature])
    group_dict[name] = df

In [20]:
for group in group_dict:
    group_dict[group]['last_month'] = group_dict[group].Value.shift(1)
    group_dict[group]['two_months_ago'] = group_dict[group].Value.shift(2)
    group_dict[group]['next_month'] = group_dict[group].Value.shift(-1)
    group_dict[group]['two_months_later'] = group_dict[group].Value.shift(-2) 
    group_dict[group] = group_dict[group].iloc[2:len(group_dict[group].Value)-2, :]

In [21]:
for group in group_dict:
    group_dict[group].to_csv('./data/grouped_' + str(group) + '.csv')